In [12]:
# Imports and Dataloading

import pandas as pd
import joblib
from sklearn.model_selection import GroupKFold, GridSearchCV, RandomizedSearchCV, cross_val_score
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.ensemble import RandomForestClassifier
from sklearn.base import BaseEstimator, TransformerMixin
from scikeras.wrappers import KerasClassifier
from scipy.stats import wilcoxon
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv1D, Flatten, Dropout, BatchNormalization
from tensorflow.keras.optimizers import Adam




df = pd.read_csv("../data/data_derivatives/features_within_subject.csv")

In [ ]:
# Re-mapping sedation level 
SEDATION_MAP = {"Baseline": 1, "Mild": 2, "Moderate": 3}
df = df[df["SedationLabel"].isin(SEDATION_MAP)]  # remove Recovery if any
df["SedationLevel"] = df["SedationLabel"].map(SEDATION_MAP)


In [ ]:
# 3. Features and target definition

exclude = [
    "Subject", "SedationLabel", "Band",
    "Propofol_ugL", "RT_ms", "Correct"
]

# Exclude _delta versions of non features
exclude += [x + "_delta" for x in exclude]

feature_cols = [
    c for c in df.columns
    if c not in exclude + ["SedationLevel"]
]

X = df[feature_cols]
y = df["SedationLevel"]

In [ ]:
# Define groups to avoid leakage
groups = df["Subject"].values
gkf = GroupKFold(n_splits=5)

In [ ]:
# Define RF pipeline parameters
pipe = Pipeline([
    ("scaler", StandardScaler()),
    ("pca", PCA()),
    ("rf", RandomForestClassifier(random_state=42, n_jobs=-1))
])


In [ ]:
# Define Parameter grid for hyperparameter optimization

param_grid = {
    "pca__n_components": [0.90, 0.95, 0.99],
    "rf__n_estimators": [200, 400, 800],
    "rf__max_depth": [None, 5, 10],
    "rf__min_samples_leaf": [1, 2, 5],
    "rf__max_features": ["sqrt", "log2", 0.5],
    "rf__class_weight": ["balanced", None],
}


grid = GridSearchCV(
    estimator=pipe,
    param_grid=param_grid,
    cv=gkf,
    scoring="accuracy",
    n_jobs=-1,
    verbose=2
)



In [ ]:
# Run search and print results

grid.fit(X, y, groups=groups)

print("\n BEST RESULT")
print("Best CV Accuracy:", f"{grid.best_score_:.3f}")
print("Best Params:")
for k,v in grid.best_params_.items():
    print(f"  {k}: {v}")

# Save best RF
best_rf = grid.best_estimator_
joblib.dump(best_rf, "best_rf.joblib")

# Evaluation of Random Forest

In [ ]:
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from sklearn.model_selection import cross_val_predict
import matplotlib.pyplot as plt

# Group-aware CV predictions
y_pred = cross_val_predict(best_rf, X, y, cv=gkf, groups=groups)

# Confusion matrix
cm = confusion_matrix(y, y_pred)

print(cm)

# nicer plot
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=["Baseline", "Mild", "Moderate"])
disp.plot(cmap="Blues")
plt.title("Group-aware Confusion Matrix (RF)")
plt.show()


In [ ]:
# Feature importance for best RF

pca = best_rf.named_steps["pca"]
rf = best_rf.named_steps["rf"]
import numpy as np
import pandas as pd

# Get PCA components (each component is a vector over original features)
components = pca.components_

# RF gives importances in PCA component space
rf_importances = rf.feature_importances_

# Reproject to original feature space
feature_importances = np.dot(rf_importances, components)

fi_series = pd.Series(feature_importances, index=feature_cols)
fi_sorted = fi_series.sort_values(ascending=False)
print(fi_sorted.head(5))


# CNN Model training

In [13]:
# define helper functions

def create_cnn_model(
    filters=16,
    kernel_size=3,
    hidden_units=32,
    dropout=0.2,
    learning_rate=1e-3 
):
    model = Sequential()
    model.add(Conv1D(filters=filters,
                     kernel_size=kernel_size,
                     activation="relu",
                     input_shape=(X.shape[1], 1)))
    model.add(BatchNormalization())
    model.add(Flatten())
    model.add(Dense(hidden_units, activation="relu"))
    model.add(Dropout(dropout))
    model.add(Dense(3, activation="softmax"))
    model.compile(
        loss="sparse_categorical_crossentropy",
        optimizer=Adam(learning_rate=learning_rate),
        metrics=["accuracy"]
    )
    return model


class ReshapeToCNN(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):
        return self
    def transform(self, X):
        return np.expand_dims(X, axis=-1)



In [15]:
# reshape input for CNN: (n_samples, n_features, 1), 1 because 3D input is expected
X_cnn = np.expand_dims(X, axis=-1)

cnn = KerasClassifier(build_fn=create_cnn_model, verbose=0)

In [16]:
# Define Parameter grid for hyperparameter optimization

param_dist = {
    "cnn__model__filters": [8, 16, 32],
    "cnn__model__kernel_size": [2, 3, 5],
    "cnn__model__hidden_units": [32, 64],
    "cnn__model__dropout": [0.1, 0.3],
    "cnn__model__learning_rate": [1e-2, 1e-3], 
    "cnn__batch_size": [16, 32],
    "cnn__epochs": [30, 60]
}

In [17]:
# define pipeline
pipe = Pipeline([
    ("scaler", StandardScaler()),
    ("reshape", ReshapeToCNN()),
    ("cnn", cnn)
])


gkf = GroupKFold(n_splits=5)

search = RandomizedSearchCV(
    pipe,
    param_distributions=param_dist,
    n_iter=20,  # fewer for speed
    cv=gkf,
    scoring="accuracy",
    verbose=2,
    n_jobs=1,  # must be 1 for Keras
)


In [18]:
# Run pipe and print results
search.fit(X, y, groups=groups)

print("Best CNN params:", search.best_params_)
print("Best CV accuracy:", search.best_score_)
best_cnn = search.best_estimator_


Fitting 5 folds for each of 20 candidates, totalling 100 fits


/Users/I743312/Library/Python/3.10/lib/python/site-packages/scikeras/wrappers.py:925: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)
/Users/I743312/Library/Python/3.10/lib/python/site-packages/keras/src/layers/convolutional/base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


[CV] END cnn__batch_size=16, cnn__epochs=30, cnn__model__dropout=0.3, cnn__model__filters=16, cnn__model__hidden_units=64, cnn__model__kernel_size=2, cnn__model__learning_rate=0.001; total time=   2.2s


/Users/I743312/Library/Python/3.10/lib/python/site-packages/scikeras/wrappers.py:925: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)
/Users/I743312/Library/Python/3.10/lib/python/site-packages/keras/src/layers/convolutional/base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


[CV] END cnn__batch_size=16, cnn__epochs=30, cnn__model__dropout=0.3, cnn__model__filters=16, cnn__model__hidden_units=64, cnn__model__kernel_size=2, cnn__model__learning_rate=0.001; total time=   1.9s


...

[CV] END cnn__batch_size=32, cnn__epochs=30, cnn__model__dropout=0.3, cnn__model__filters=32, cnn__model__hidden_units=32, cnn__model__kernel_size=3, cnn__model__learning_rate=0.01; total time=   1.8s


/Users/I743312/Library/Python/3.10/lib/python/site-packages/scikeras/wrappers.py:925: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)
/Users/I743312/Library/Python/3.10/lib/python/site-packages/keras/src/layers/convolutional/base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


[CV] END cnn__batch_size=32, cnn__epochs=30, cnn__model__dropout=0.3, cnn__model__filters=32, cnn__model__hidden_units=32, cnn__model__kernel_size=3, cnn__model__learning_rate=0.01; total time=   1.7s


/Users/I743312/Library/Python/3.10/lib/python/site-packages/scikeras/wrappers.py:925: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)
/Users/I743312/Library/Python/3.10/lib/python/site-packages/keras/src/layers/convolutional/base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Best CNN params: {'cnn__model__learning_rate': 0.001, 'cnn__model__kernel_size': 3, 'cnn__model__hidden_units': 32, 'cnn__model__filters': 16, 'cnn__model__dropout': 0.3, 'cnn__epochs': 60, 'cnn__batch_size': 16}
Best CV accuracy: 0.7223163841807909


In [ ]:
# Save best CNN model
best_cnn.named_steps["cnn"].model_.save("cnn_model.keras")

# Evaluation of Convulational Neural Network

In [ ]:
# Group-aware CV predictions
y_pred = cross_val_predict(best_cnn, X, y, cv=gkf, groups=groups)

# Confusion matrix
cm = confusion_matrix(y, y_pred)

print(cm)

# Plot (same style as RF)
disp = ConfusionMatrixDisplay(
    confusion_matrix=cm,
    display_labels=["Baseline", "Mild", "Moderate"]
)
disp.plot(cmap="Blues")
plt.title("Group-aware Confusion Matrix (CNN)")
plt.show()

# Statistical Analyses

In [ ]:
# Run and save GroupKFold for RF and CNN

gkf = GroupKFold(n_splits=5)

##rf_scores = cross_val_score(best_rf, X, y, cv=gkf, groups=groups, scoring="accuracy")
print("RF fold accuracies:", rf_scores)

cnn_scores = cross_val_score(best_cnn, X, y, cv=gkf, groups=groups, scoring="accuracy")
print("CNN fold accuracies:", cnn_scores)

#print(f"\nMean RF accuracy:  {rf_scores.mean():.3f}")
print(f"Mean CNN accuracy: {cnn_scores.mean():.3f}")

#np.save("../data/models/rf_cv_scores.npy", rf_scores)
np.save("../data/models/cnn_cv_scores.npy", cnn_scores)



In [ ]:
# Run Wilcoxon signed-rank
stat, p = wilcoxon(rf_scores, cnn_scores)

print(f"Statistic: {stat:.4f}")
print(f"p-value:   {p:.6f}")

# Visualizations

In [ ]:
# Plot Line Chart

plt.figure(figsize=(6,4))
plt.plot(rf_scores, marker='o', label="RF")
plt.plot(cnn_scores, marker='o', label="CNN")
plt.xticks(range(len(rf_scores)), [f"Fold {i+1}" for i in range(len(rf_scores))])
plt.ylabel("Accuracy")
plt.title("GroupKFold Accuracy per Fold")
plt.ylim(0,1)
plt.legend()
plt.grid(alpha=0.3)
plt.show()


In [ ]:
# Plot Bar Chart
mean_rf = rf_scores.mean()
mean_cnn = cnn_scores.mean()
sd_rf = rf_scores.std()
sd_cnn = cnn_scores.std()

plt.figure(figsize=(5,4))

plt.bar(
    ["RF", "CNN"],
    [mean_rf, mean_cnn],
    yerr=[sd_rf, sd_cnn],
    capsize=5,
    color=["#3F61C4", "#FFDC00"]
)

plt.ylabel("Accuracy")
plt.title("Model Performance Comparison")

plt.ylim(0.5, 0.9) 
plt.tight_layout()
plt.show()
